In [1]:
import itertools

import numpy as np
from pyomo.environ import *
from pyomo.environ import value as pyoval
from pyomo.environ import Binary, NonNegativeReals
from typing import Tuple, Union
from collections import defaultdict

In [2]:
def find_all_active_sets(df_dz: np.ndarray, m=None, integer_cuts=None, active_sets=defaultdict(dict), iter_count=0) -> list:
    if not isinstance(m, ConcreteModel):
        a = np.where(df_dz > 0, 1, np.where(df_dz < 0, -1, 0))
        n_c = a.shape[0]
        m = ConcreteModel()
        
        m.ai = RangeSet(n_c)
        m.aj = RangeSet(a.shape[1])
        m.J_blocks = Set(initialize=list(range(1, n_c+1)))
        J_pos = {i:[j+1 for j in np.where(a[i-1] > 0)[0]] for i in m.J_blocks}
        m.J_pos = Set(m.J_blocks, initialize=J_pos)
        J_neg = {i:[j+1 for j in np.where(a[i-1] < 0)[0]] for i in m.J_blocks}
        m.J_neg = Set(m.J_blocks, initialize=J_neg)
        
        m.y = Var(m.aj, domain=Binary)
        
        def J_pos_rule(instance, i):
            return sum(a[i-1][j-1]*instance.y[j] for j in instance.J_pos[i]) >= 1
        m.J_pos_constraint = Constraint(m.ai, rule=J_pos_rule)
        
        def J_neg_rule(instance, i):
            return sum(-a[i-1][j-1]*instance.y[j] for j in instance.J_neg[i]) >= 1
        m.J_neg_constraint = Constraint(m.ai, rule=J_neg_rule)
        
        def binary_sum_rule(instance):
            return sum(instance.y[j] for j in instance.aj) == n_c + 1
        m.binary_sum_constraint = Constraint(rule=binary_sum_rule)
    
        m.objective = Objective(expr=0, sense=minimize)
    else:
        n_c = len(m.ai)
        
    if integer_cuts:
        if hasattr(m, 'cuts'):
            m.del_component(m.cuts)
        m.cuts = ConstraintList()
        for cut_expr in integer_cuts:
            m.cuts.add(cut_expr)
    
    solver = SolverFactory('gurobi')
    results = solver.solve(m)
    iter_count+=1

    if results.solver.termination_condition == TerminationCondition.infeasible:
        return active_sets
    
    new_cut = (sum(m.y[j] for j in m.aj if pyoval(m.y[j]) == 1) - sum(m.y[j] for j in m.aj if pyoval(m.y[j]) == 0) <= n_c)
    active_sets[iter_count] = [j for j in m.aj if pyoval(m.y[j]) == 1]
    
    print(f'Cut added for iteration {iter_count}: {new_cut}')

    new_cuts = list(integer_cuts) if integer_cuts else list()
    new_cuts.append(new_cut)

    return find_all_active_sets(df_dz=df_dz, m=m, integer_cuts=new_cuts, active_sets=active_sets, iter_count=iter_count)

In [3]:
# EXAMPLE 1 Pistikopoulos and Grossmann (1988): https://www.sciencedirect.com/science/article/pii/0098135488800103
theta_nominal = [2]
theta_deviations = [(2,2)]

A = np.array([
    [1, -3, 1, -1],
    [0, 1, -1, -1/3],
    [-1, 0, 1 ,1]
])

b = np.array([0, -1/3, 1])

d = np.array([3, 1])

In [4]:
n_d, n_t = 2, 1
n_c = A.shape[1] - (n_d+n_t)

In [5]:
df_dz = A[:, n_d:n_d+n_c].T

In [6]:
active_set_dict = find_all_active_sets(df_dz=df_dz)

Cut added for iteration 1: y[2] + y[3] - y[1]  <=  1
Cut added for iteration 2: y[1] + y[2] - y[3]  <=  1
model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.


In [7]:
active_set_dict

defaultdict(dict, {1: [2, 3], 2: [1, 2]})